In [1]:
import numpy as np
from scipy import sparse as sp
import scipy.linalg as la
import pandas as pd

**Problem 1**

In [2]:
def adj_mat(filename, N):
    A = sp.dok_matrix((N, N))
    with open('matrix.txt', 'r') as myFile:
        try:
            for line in myFile:
                numbers = line.strip().split()
                if len(numbers) > 2:
                    continue
                else:
                    A[int(numbers[0]), int(numbers[1])] = 1
        except:
            raise ValueError('File error')
        finally:
            myFile.close()
    return A

In [3]:
A = adj_mat('matrix.txt', 8)
print(A)

  (0, 7)	1.0
  (1, 0)	1.0
  (3, 0)	1.0
  (3, 2)	1.0
  (3, 6)	1.0
  (4, 0)	1.0
  (4, 5)	1.0
  (4, 6)	1.0
  (5, 0)	1.0
  (5, 6)	1.0
  (6, 0)	1.0
  (7, 0)	1.0


In [4]:
print(A.toarray())

[[0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1. 1. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]]


**Problem 2**

In [5]:
def compKmat(A):
    for i in range(len(A)):
        if sum(A[i, :]) == 0:
            A[i, :] = 1
    # Compute the diagonal matrix D
    D = np.zeros((len(A), len(A)))
    for i in range(len(A)):
        D[i, i] = sum(A[i , :])
    K = (np.linalg.inv(D) @ A).T
    return K

In [6]:
K = compKmat(A.toarray())
print(K)

[[0.         1.         0.125      0.33333333 0.33333333 0.5
  1.         1.        ]
 [0.         0.         0.125      0.         0.         0.
  0.         0.        ]
 [0.         0.         0.125      0.33333333 0.         0.
  0.         0.        ]
 [0.         0.         0.125      0.         0.         0.
  0.         0.        ]
 [0.         0.         0.125      0.         0.         0.
  0.         0.        ]
 [0.         0.         0.125      0.         0.33333333 0.
  0.         0.        ]
 [0.         0.         0.125      0.33333333 0.33333333 0.5
  0.         0.        ]
 [1.         0.         0.125      0.         0.         0.
  0.         0.        ]]


**Problem 3**

In [7]:
def itermeth(A, N=None, d=.85, tol=1e-05):
    if N:
        A = A[:N, :N]
        p_vector = np.random.random(N)
    else:
        N = len(A)
        p_vector = np.random.random(N)
    K = compKmat(A)
    ɛ = tol + 1
    while ɛ > tol:
        p_vector2 = d * (K @ p_vector) + ((1 - d) / N) * np.ones(N)
        ɛ = la.norm(p_vector2 - p_vector)
        p_vector = p_vector2
    return p_vector

In [8]:
t = itermeth(A.toarray())
t

array([0.43872245, 0.02171029, 0.02786154, 0.02171029, 0.02171029,
       0.02786154, 0.04585394, 0.39462494])

**Problem 4**

In [9]:
def eigmeth(A, N=None, d=.85):
    if N:
        A = A[:N, :N]
    else:
        N = len(A)
    K = compKmat(A)
    B = d * K + ((1 - d) / N) * np.ones((N, N))
    eigs, vecs = la.eig(B)
    
    return(vecs[:,0] / np.sum(vecs[:,0]))

In [10]:
eigmeth(A.toarray())

array([0.43869288, 0.02171029, 0.02786154, 0.02171029, 0.02171029,
       0.02786154, 0.04585394, 0.39459924])

**Problem 5**

In [11]:
ncaa2013 = pd.read_csv('ncaa2013.csv', header = 0).as_matrix()
m, n = ncaa2013.shape
teams, index = np.unique(ncaa2013, return_inverse = True)
index = index.reshape((m, n))

t = len(teams)
adj_mat = np.zeros((t,t))

for i in range(m):
    adj_mat[index[i, 1], index[i, 0]] = 1

ranks = itermeth(adj_mat, d = 0.7)
np.argsort(ranks.T)
ranks = teams[np.argsort(ranks.T)].T[::-1]

print('Top five ranked teams:')
for i in range(5):
    print(ranks[i], end=" ")

Top five ranked teams:
Duke Butler Louisville Illinois Indiana 

C:\Users\danie\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
